In [1]:
# packages
from ortools.sat.python import cp_model
from pyomo.environ import *
import pandas as pd
import numpy as np
from bokeh.io import show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

<font size=16>Homework 07: Constraint Programming</font>

# **HW7.1** - Textbook Problem 12.9-2

Consider the following problem:

Maximize $Z = 5x_1 - x_1^2 + 8x_2 - x_2^2 + 10x_3 - x_3^2 + 15 x_4 - x_4^2 + 20 x_5 - x_5^2$

subject to 

$ x_1 \in \{3,6,12\}, x_2 \in \{3,6\}, x_3 \in \{3,6,9,12\}, x_4 \in \{6,12\}, x_5 \in \{9,12,15,18\}$

$x_1, x_2, x_3, x_4, x_5$ must all be different 

$x_1 + x_3 + x_4 \leq 25$

**HW7.1a** - 
Without doing any optimization, use CP-SAT to make a list of all feasible solutions.  Your code should be easily generalizable (use an abstract approach).  

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [2]:
# CP-SAT Find all feasible solutions

# solution printer
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0

    def on_solution_callback(self):
        self.__solution_count += 1
        for v in self.__variables:
            print(f'{v} = {self.Value(v)}', end = ' ')
        print()

    def solution_count(self):
        return self.__solution_count
    
# Model
model = cp_model.CpModel()

# Variables
sets = {'x1':[3,6,12],'x2':[3,6],'x3':[3,6,9,12],'x4':[6,12],'x5':[9,12,15,18]}
dvars = [model.NewIntVarFromDomain(cp_model.Domain.FromValues(sets[v]),v) for v in sets.keys()]

# Constraints
model.Add(dvars[0] + dvars[2] + dvars[3] <= 25)
model.AddAllDifferent(dvars)

# Solve
solver = cp_model.CpSolver()  
solution_printer = VarArraySolutionPrinter(dvars)
status = solver.SearchForAllSolutions(model, solution_printer)

print(f'Status = {solver.StatusName(status)}')
print(f'Number of solutions found: {solution_printer.solution_count()}')

x1 = 3 x2 = 6 x3 = 9 x4 = 12 x5 = 15 
x1 = 3 x2 = 6 x3 = 9 x4 = 12 x5 = 18 
Status = OPTIMAL
Number of solutions found: 2


**HW7.1b** Now use CP-SAT to solve the optimization problem.  Your code should be easily generalizable (use an abstract approach).  

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [3]:
# CP-SAT Optimization

# Create the model.
model = cp_model.CpModel()

# Variables
x_dict = {'x1':[3,6,12],'x2':[3,6],'x3':[3,6,9,12],'x4':[6,12],'x5':[9,12,15,18]}
x = [model.NewIntVarFromDomain(cp_model.Domain.FromValues(x_dict[v]),v) for v in x_dict.keys()]

minx = min([min(x_dict[v]) for v in x_dict.keys()])
maxx = max([max(x_dict[v]) for v in x_dict.keys()])
xsq = [model.NewIntVar(minx**2,maxx**2,f'{v}sq') for v in x_dict.keys()]

# Constraints
model.Add(x[0] + x[2] + x[3] <= 25)
model.AddAllDifferent(x)
for i in range(len(x)):
    model.AddMultiplicationEquality( xsq[i], [ x[i], x[i] ] )

# Objective Function
coef = [5, 8, 10, 15, 20]
model.Maximize( sum( coef[i]*x[i] - xsq[i] for i in range(len(x))) )

# Solve
solver = cp_model.CpSolver()
status = solver.Solve(model)

# Display
print('Maximum of objective function: %i' % solver.ObjectiveValue())
print()
for v in x:
    print(f'{v} = {solver.Value(v)}')

Maximum of objective function: 138

x1 = 3
x2 = 6
x3 = 9
x4 = 12
x5 = 15


# **HW7.2** - Assignment Problem

This problem is based on textbook problem 9.1-7 

The Move-It Company has two plants producing forklift trucks that then are shipped to three distribution centers. The production costs are the same at the two plants, and the cost of shipping for each truck is shown for each combination of plant and distribution center: 

<img src="./images/forklift_table.png" width=400>

A total of 60 forklift trucks are produced and shipped per week. Each plant can produce and ship any amount up to a maximum of 50 trucks per week, so there is considerable flexibility on how to divide the total production between the two plants so as to reduce shipping costs. However, each distribution center must receive exactly 20 trucks per week.

The objective of management is to determine how many forklift trucks should be produced at each plant, and then what the overall shipping pattern should be to minimize total shipping cost.

We are going to solve this two different ways:

**HW7.2a** -  *Allow product splitting.*  Each distribution center can receive forklift trucks from both plants (it's possible distribution center 1 gets 10 from A and 10 from B).    This is not a CP-SAT problem, **use Pyomo to solve this problem**.  You may find code from earlier lessons to be helpful.  Note that the total supply is greater than the total demand so you'll to include a dummy distribution center that receives the excess supply to turn this into a balanced transportation problem.  
<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [4]:
# Pyomo Forklift Shipping

costs = [[600, 700, 400],[700, 800, 500]]
plants = ['A', 'B']
dist_ctrs = ['1', '2', '3']
num_ctrs = len(dist_ctrs)
num_plants = len(plants)
ship_costs = {plants[p]: {dist_ctrs[d]: costs[p][d] for d in range(num_ctrs)} for p in range(num_plants)}

# Model
model = ConcreteModel(name="7.2a")

# Decisions Variables
model.ship = Var(plants, dist_ctrs, domain=NonNegativeReals)

# Objective
model.total_cost = Objective(expr=sum(ship_costs[p][d] * model.ship[p, d] for p in plants for d in dist_ctrs), sense=minimize)

# Contraints
model.constraints = ConstraintList()

# 50 forklifts per plant
for p in plants:
    model.constraints.add(sum(model.ship[p, d] for d in dist_ctrs) <= 50)

# 20 forklifts per dist_ctr
for d in dist_ctrs:
    model.constraints.add(sum(model.ship[p, d] for p in plants) == 20)
    
# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Display
print(f"Minimum costs: ${model.total_cost():,.2f}\n")
print("Assignments:")
assignments = pd.DataFrame([[model.ship[p, d]() for d in dist_ctrs] for p in plants], index = plants, columns = dist_ctrs)
assignments

Minimum costs: $35,000.00

Assignments:


,1,2,3
A,20.0,20.0,10.0
B,0.0,0.0,10.0


**HW7.2b** - *No product splitting allowed* . Each distribution center can receive forklift trucks from only one plant (to lower administrative and other hidden costs).   So one plant sends two shipments of 20 to each of two distribution centers, while the other plant sends one shipment of 20 to the remaining distribution center.

Follow the "Formulation of option 2" on page 354 to formulate this as an assignment problem, then use CP-SAT with Element and All-Different constraints to solve this assignment problem to find the minimum cost.  The minimum cost with no-splitting allowed should be larger than the minimum cost when splitting is allowed.  Your code should be easily generalizable.  Make no assumptions about the quantity shipped from any plant.  Your code should still work if the cost table is changed.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [5]:
# CP-SAT Forklift Shipping - no product splitting

# problem data
M = 0
costs = [[600, 700, 400, M],[600, 700, 400, M],[700, 800, 500, M],[700, 800, 500, M]]
plants = ['A1', 'A2', 'B1', 'B2']
dist_ctrs = ['1', '2', '3', '-']
num_ctrs = len(dist_ctrs)
num_plants = len(plants)
shipment_size = 20
#ship_costs = {plants[p]: {dist_ctrs[d]: costs[p][d] for d in range(num_ctrs)} for p in range(num_plants)}

# Create the model.
model = cp_model.CpModel()

# Variables
assign = [model.NewIntVar(0, num_ctrs - 1, plants[p]) for p in range(num_plants)]

#get the maximum cost from the cost_table for the top of our intvar range
max_cost = max(list(map(max, costs)))
cost = [model.NewIntVar(0, max_cost, f'cost{i}') for i in range(num_plants)]

# Constraints
model.AddAllDifferent(assign)

for i in range(num_plants):
    model.AddElement(assign[i], costs[i], cost[i]) #cost[i]= cost_table[i][assign[i]] 


    
# Objective
model.Minimize(sum(cost))

# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    
    print(f'Lowest Possible Cost: ${solver.ObjectiveValue() * shipment_size:,.2f}')
    print()
    print('Assignments and associated costs:\n')
    cost_assigns = pd.DataFrame(0, index=plants, columns=dist_ctrs)
    for i in range(num_plants):
        
        cost_assigns.iloc[i, solver.Value(assign[i])] = solver.Value(cost[i]) * shipment_size
    
    print(cost_assigns)

Lowest Possible Cost: $36,000.00

Assignments and associated costs:

        1      2      3  -
A1      0  14000      0  0
A2  12000      0      0  0
B1      0      0  10000  0
B2      0      0      0  0


# **HW7.3** - Different Assignment-like Problem

A college student has 7 days remaining before final examinations begin in her four courses, and she wants to allocate this study time as effectively as possible. She needs at least 1 day on each course, and she likes to concentrate on just one course each day, so she wants to allocate 1, 2, 3, or 4 days to each course. Having recently taken an OR course, she decides to use dynamic programming to make these allocations to maximize the total grade points to be obtained from the four courses. She estimates that the alternative allocations for each course would yield the number of grade points shown in the following table:

<img src="./images/grades_table.png" width=400>

Use a CP-SAT constraint programming approach with the Element constraint and other appropriate constraints to maximize the total grade points. Your code should be easily generalizable.  Does it make sense to use `all-different` here?  Are you making assignments in each row or in each column? (These questions are to help you think about the problem and don't require written answers.)

<font color = "blue"> *** 10 points -  answer in cell below *** (don't delete this cell) </font>

In [83]:
# CP-SAT Maximize GPA

courses = ['Course 1', 'Course 2', 'Course 3', 'Course 4']
days = ['1', '2', '3', '4']
est_gp = [[3, 5, 6, 7], [5, 5, 6, 9], [2, 4, 7, 8], [6, 7, 9, 9]] # days as columns, courses as rows
#est_gp = [[3, 5, 2, 6], [5, 5, 4, 7], [6, 6, 7, 9], [7, 9, 8, 9]] # courses as columns, days as rows
num_days = len(days)
num_courses = len(courses)
max_points = max([j for i in est_gp for j in i])
study_days_available = 7


# Create the model.
model = cp_model.CpModel()

assign = [model.NewIntVar(0, num_days - 1, courses[i]) for i in range(num_courses)]

gp = [model.NewIntVar(0, max_points, f'gp{i}') for i in range(num_days)]

model.Add(sum(assign[i] for i in range(num_courses)) <= (study_days_available - num_days))

for i in range(num_courses):
    model.AddElement(assign[i], est_gp[i], gp[i])

model.Maximize(sum(gp))



# Creates a solver and solves the model.
solver = cp_model.CpSolver()
status = solver.Solve(model)

if status == cp_model.OPTIMAL:
    print(
        f'Expected grade point improvement: {solver.ObjectiveValue()}')
    print()
    course_assigns = pd.DataFrame(0, index=courses, columns=days)
    for i in range(num_days):
        course_assigns.iloc[ i, solver.Value(assign[i]) ] = solver.Value(gp[i])
    print('Number of days assigned and grade points gained:\n')
    print(course_assigns)
 


Expected grade point improvement: 23.0

Number of days assigned and grade points gained:

          1  2  3  4
Course 1  0  5  0  0
Course 2  5  0  0  0
Course 3  0  0  7  0
Course 4  6  0  0  0


# **HW7.4** - Scheduling

Use the generalizable CP-SAT approach to scheduling shown in the lesson to find a schedule of minimum length for the Reliable Construction Company project with activities A-N shown in the table below:

<img src="./images/reliable_table.png" width=600>

This problem is discussed in a supplemental textbook chapter which we've included in the folder with this notebook if you want to know more about it.

Display both the minimized length and the optimal schedule in both text and with a Gantt chart.

<font color = "blue"> *** 8 points -  answer in cell below *** (don't delete this cell) </font>

In [7]:
# generalizable schedule code

task_duration_dict = {
    'excavate': 2,
    'foundation': 4,
    'rough wall': 10,
    'roof': 6,
    'exterior plumbing': 4,
    'interior plumbing': 5,
    'exterior siding': 7,
    'exterior painting': 9,
    'electrical': 7,
    'wallboard': 8,
    'flooring': 4,
    'interior painting': 5,
    'exterior fixtures': 2,
    'interior fixtures': 6
}
task_names = list(task_duration_dict.keys())
num_tasks = len(task_names)
durations = list(task_duration_dict.values())

# for each task we have a list of tasks that must go after
# task:['these','tasks','after']
precedence_dict = {
    'excavate': ['foundation'],
    'foundation': ['rough wall'],
    'rough wall': ['roof', 'exterior plumbing', 'electrical'],
    'roof': ['exterior siding'],
    'exterior plumbing': ['interior plumbing', 'exterior painting'],
    'interior plumbing': ['wallboard'],
    'exterior siding': ['exterior painting'],
    'exterior painting': ['exterior fixtures'],
    'electrical': ['wallboard'],
    'wallboard': ['flooring', 'interior painting'],
    'flooring': ['interior fixtures'],
    'interior painting': ['interior fixtures']
}

task_name_to_number_dict = dict(zip(task_names, np.arange(0, num_tasks)))

horizon = sum(task_duration_dict.values())

model = cp_model.CpModel()

start_vars = [
    model.NewIntVar(0, horizon, name=f'start_{t}') for t in task_names
]
end_vars = [model.NewIntVar(0, horizon, name=f'end_{t}') for t in task_names]

# the `NewIntervalVar` are both variables and constraints, the internally enforce that start + duration = end
intervals = [
    model.NewIntervalVar(start_vars[i],
                         durations[i],
                         end_vars[i],
                         name=f'interval_{task_names[i]}')
    for i in range(num_tasks)
]

# precedence constraints
for before in list(precedence_dict.keys()):
    for after in precedence_dict[before]:
        before_index = task_name_to_number_dict[before]
        after_index = task_name_to_number_dict[after]
        model.Add(end_vars[before_index] <= start_vars[after_index])

obj_var = model.NewIntVar(0, horizon, 'largest_end_time')
model.AddMaxEquality(obj_var, end_vars)
model.Minimize(obj_var)

solver = cp_model.CpSolver()
status = solver.Solve(model)

print(f'Optimal Schedule Length: {solver.ObjectiveValue()}')
for i in range(num_tasks):
    print(
        f'{task_names[i]} start at {solver.Value(start_vars[i])} and end at {solver.Value(end_vars[i])}'
    )
    
# chart
output_notebook()

starts = [solver.Value(start_vars[i]) for i in range(num_tasks)]
ends = [solver.Value(end_vars[i]) for i in range(num_tasks)]

source = ColumnDataSource(data=dict(tasks=task_names, starts = starts, ends=ends))

p = figure(x_range=(0,solver.ObjectiveValue()), y_range=task_names, plot_height=350, title="Task Time Spans",
           toolbar_location=None, tools="")

p.hbar(y='tasks', left='starts', right='ends', height=0.9, source=source)

p.xaxis.axis_label = "Time"
p.ygrid.grid_line_color = None

show(p)

Optimal Schedule Length: 44.0
excavate start at 0 and end at 2
foundation start at 2 and end at 6
rough wall start at 6 and end at 16
roof start at 16 and end at 22
exterior plumbing start at 16 and end at 20
interior plumbing start at 20 and end at 25
exterior siding start at 22 and end at 29
exterior painting start at 29 and end at 38
electrical start at 16 and end at 23
wallboard start at 25 and end at 33
flooring start at 33 and end at 37
interior painting start at 33 and end at 38
exterior fixtures start at 38 and end at 40
interior fixtures start at 38 and end at 44


Loading BokehJS ...